In [ ]:
import pandas as pd
import requests as rq
import base64
from datetime import datetime, timedelta
import matplotlib

In [ ]:
%matplotlib inline

# Authorization header

In [ ]:
togglApiKey = ''
credential = f'{togglApiKey}:api_token'
encoded = base64.b64encode(credential.encode('ascii'))
headers = {'Authorization': f"Basic {encoded.decode('utf-8')}"}

# Projects

## Request

In [ ]:
prefix_url = 'https://www.toggl.com/api/v8/workspaces'
url = f'{prefix_url}/1825005/projects'
r = rq.get(url, headers=headers)

## Make a dataframe with result

In [ ]:
projects = pd.read_json(r.text, convert_dates = ['at','created_at'])
projects.set_index('id',inplace=True)

# Sessions

## Last week

In [ ]:
last_week = datetime.today() - timedelta(days=300)
last_week = last_week.strftime("%Y-%m-%d")
start_date = f'{last_week}T00%3A00%3A00%2B00%3A00'

## Request

In [ ]:
prefix_url = 'https://www.toggl.com/api/v8/time_entries'
url = f'{prefix_url}?start_date={start_date}'
r = rq.get(url, headers=headers)

## Make a dataframe with result

In [ ]:
sessions = pd.read_json(r.text, convert_dates = ['at','start','stop'])
sessions.set_index('at',inplace=True)

In [ ]:
merge = sessions.merge(projects[['name']], left_on='pid', right_index=True,  how='inner')

In [ ]:
merge

In [ ]:
sums = merge.duration.groupby(merge.name).sum()
sums = sums/3600

In [ ]:
def absolute_value(val):
    return val
sums.plot(kind='pie',startangle=90, legend=False, autopct=absolute_value, figsize =(15,12))

In [ ]:
sums